In [12]:
import asyncio
from copra.websocket import Channel, Client
import matplotlib.pyplot as plt
from collections import OrderedDict
from time import sleep
from dateutil import parser
import copy
import datetime
import itertools
from operator import itemgetter
import numpy as np
from dateutil.tz import tzutc
import math
import pytz
from pytz import timezone
import pickle
import pylab
from scipy.stats import probplot,expon,kstest
import matplotlib.pyplot as plt

In [21]:
def parse_file(file_name):
    raw_updates = []
    with open(file_name,"rb") as f:
        starting_time,starting_bids,starting_asks = list(pickle.load(f))
        while True:
            try:
                raw_updates.append(pickle.load(f))
            except EOFError:
                break
    
    Bids = {}
    Asks = {}    
    
    updates = []
    
    for price, amount in starting_bids:
        Bids[int(round((float(price)*100)))] = float(amount)
    for price, amount in starting_asks:
        Asks[int(round((float(price)*100)))] = float(amount)
        
    for u in raw_updates:
        price = int(round((float(u["price"])*100)))  
        side = u["side"]
        amount = float(u["amount"])
        time = parser.parse(u["time"])
        
        if side == "buy":
            change = amount - Bids.get(price,0)
        else:
            change = amount - Asks.get(price,0)
        
        updates.append({\
            "Bids": copy.copy(Bids), \
            "Asks": copy.copy(Asks), \
            "time": time, \
            "side": side, \
            "price": price, \
            "change": change            
        })
        
        if side == "buy":
            if amount == "0":
                del Bids[price]
            else:
                Bids[price] = float(amount)
        else:
            if amount == "0":
                del Asks[price]
            else:
                Asks[price] = float(amount)   

    
    return(starting_time, updates)

parsed_file = parse_file("12_25_18_data.pkl")
len(parsed_file[1])

28870

In [22]:
def shortened_updates(file_name,K):
    starting_time, updates = parse_file(file_name)
    
    res = []
    
    # Caluclate first reference price
    starting_bids = updates[0]['Bids']
    starting_asks = updates[0]['Asks']
    sorted_bids = list(reversed(sorted(starting_bids.items())))
    sorted_asks = list(sorted(starting_asks.items()))        
    best_bid = sorted_bids[0][0]
    best_ask = sorted_asks[0][0]
    if ((best_bid + best_ask) % 2) != 0:
        old_reference_price = round((best_bid+best_ask)/2, 1)
    else:
        old_reference_price = round((best_bid+best_ask+1)/2, 1)
            
    for u in updates:      
        # Find reference price
        sorted_bids = list(reversed(sorted(u['Bids'].items())))
        sorted_asks = list(sorted(u['Asks'].items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            middle = (best_bid+best_ask)/2
            if old_reference_price > middle:
                reference_price = round((best_bid+best_ask)/2 + 0.5,1)
            else:
                old_reference_price = round((best_bid+best_ask)/2 - 0.5,1)
                
        shortened_book = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
        first_bid = int(round(reference_price - 0.5))
        first_ask = int(round(reference_price + 0.5))        
        for k in range(-K,0):
            shortened_book[k] = u['Bids'].get(first_bid + k + 1,0)
        for k in range(1,K+1):
            shortened_book[k] = u['Asks'].get(first_ask + k - 1,0)
            
        # Find k from the price. Keep track of event if
        # abs(k) <= K
        price = u["price"]
        k = price - reference_price
        if k < 0:
            k = int(round(k - 0.5))
        else:
            k = int(round(k + 0.5))
        if abs(k) <= K:                 
            res.append({
                'reference_price': reference_price,
                'LOB': copy.copy(shortened_book),
                'k': k,
                'change': u['change'],
                'time': u['time']
            })
        
        old_reference_price = reference_price

    return starting_time, res

starting_time,updates = shortened_updates('12_25_18_data.pkl',10)
len(updates)

27922

In [23]:
grouped_by_time = OrderedDict([(k, list(v)) for k, v in itertools.groupby(updates, key=lambda x:x['time'])])
# Contains dictionary of time, reference price, order book, list of changes
cleaned_updates = []
for t, us in grouped_by_time.items():
    if len(us) == 1:
        u = us[0]
        cleaned_updates.append(copy.copy(u))
    else:
        new_update = {'time': t}
        us = sorted(us, key=lambda u:-abs(u['k']))
        grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(us, key=lambda u:u['k']))
        reference_k = list(grouped_by_k.keys())[0]
        new_update['reference_price'] = grouped_by_k[reference_k][0]['reference_price']
        new_update['LOB'] = copy.copy(grouped_by_k[reference_k][0]['LOB'])
        events = []
        for k in grouped_by_k:
            combined_change = 0
            for u in grouped_by_k[k]:
                combined_change = combined_change + u['change']
            events.append((k,combined_change))
        for k,change in events:
            new_update = copy.deepcopy(new_update)
            new_update['k'] = k
            new_update['change'] = change
            cleaned_updates.append(new_update)

cleaned_updates
len(cleaned_updates)

27720

In [24]:
combined_updates = []
i = 0
while i < len(cleaned_updates):
    reference_price = cleaned_updates[i]['reference_price']
    j = i
    updates_at_reference = []
    while (j < len(cleaned_updates)) and (cleaned_updates[j]['reference_price'] == reference_price):
        updates_at_reference.append(cleaned_updates[j])
        j += 1
    updates_at_reference = sorted(updates_at_reference,key=lambda u:u['k'])
    grouped_by_k = OrderedDict((k, list(v)) for k, v in itertools.groupby(updates_at_reference, key=lambda u:u['k']))
    for k,us in grouped_by_k.items():
        us = copy.deepcopy(sorted(us,key=lambda u:u['time']))
        keep_index = [True for u in us]
        for m in reversed(range(1,len(us))):
            quick_same_order = (us[m]['time'] - us[m-1]['time']).total_seconds() < 0.01
            same_sign = (us[m]['change'] * us[m-1]['change']) > 0
            if quick_same_order and same_sign:
                us[m-1]['change'] += us[m]['change']
                keep_index[m] = False
        for (u,keep) in zip(us, keep_index):
            if keep:
                combined_updates.append(u)
    i = j + 1
    
combined_updates = sorted(combined_updates,key=lambda u:u['time'])
len(combined_updates)

26870

In [169]:
from datetime import timedelta
n = 1500
ending_time = updates[-1]['time'] + timedelta(seconds=1)
ending_time + timedelta(seconds = 1)
time_period_length = (ending_time - starting_time).total_seconds()

times = [starting_time + timedelta(seconds=i*time_period_length/n) for i in range(1,n+1)]

changes_at_each_k = OrderedDict([(k,[0 for _ in range(n)]) for k in range(-K,K+1) if k != 0])

i = 0
for u in combined_updates:
    if u['change'] < 0:
        changes_at_each_k[u['k']][i] -= 1
    else:
        changes_at_each_k[u['k']][i] += 1
        
    if u['time'] > times[i]:
        i += 1

In [170]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame(changes_at_each_k)
correlations = df.corr().astype('object')

In [204]:
from scipy.stats.stats import pearsonr
significances = correlations.copy()
for k1 in changes_at_each_k:
    for k2 in changes_at_each_k:
        res = pearsonr(changes_at_each_k[k1],changes_at_each_k[k2])
        significances[k1][k2] = res
        
significances = significances.loc[-5:5,-5:5]
significances

,-5,-4,-3,-2,-1,1,2,3,4,5
-5,"(1.0, 0.0)","(0.0668729137474, 0.00957759295158)","(-0.155820752264, 1.30342842138e-09)","(-0.0684757900826, 0.00797894730348)","(-0.0182531417751, 0.479932801952)","(-0.0113213297804, 0.661298048381)","(0.0190959014249, 0.459887620012)","(0.0342702843992, 0.184652432528)","(-0.0342136900742, 0.18537900548)","(0.0200705424213, 0.43730098135)"
-4,"(0.0668729137474, 0.00957759295158)","(1.0, 0.0)","(-0.0909426380843, 0.000420985129583)","(-0.0874806724315, 0.000694372565144)","(0.0387797081649, 0.133292399762)","(-0.0222051877932, 0.390124151029)","(0.00970826453598, 0.707143753101)","(0.00380659520783, 0.882890159293)","(0.0101913679454, 0.693292259162)","(0.0121418442589, 0.638442969739)"
-3,"(-0.155820752264, 1.30342842138e-09)","(-0.0909426380843, 0.000420985129583)","(1.0, 0.0)","(-0.101417770869, 8.32771720005e-05)","(-0.0427701784826, 0.0977504439228)","(-0.0843919117177, 0.00106941725535)","(-0.0474923157426, 0.0659337607514)","(0.0246731630186, 0.339609945091)","(0.0436912904085, 0.0907311410579)","(-0.0400330449048, 0.121189076621)"
-2,"(-0.0684757900826, 0.00797894730348)","(-0.0874806724315, 0.000694372565144)","(-0.101417770869, 8.32771720005e-05)","(1.0, 0.0)","(-0.160060028807, 4.54197896116e-10)","(-0.0755602098864, 0.00340951029862)","(-0.0437587560729, 0.0902333647191)","(-0.0416318930665, 0.107016161819)","(0.0614913850597, 0.0172273224942)","(-0.00116456763245, 0.964054761673)"
-1,"(-0.0182531417751, 0.479932801952)","(0.0387797081649, 0.133292399762)","(-0.0427701784826, 0.0977504439228)","(-0.160060028807, 4.54197896116e-10)","(1.0, 0.0)","(-0.0202991242145, 0.432097621595)","(-0.183851398327, 7.1860046752e-13)","(-0.0585099225782, 0.0234433051583)","(-0.0504282614334, 0.0508560550686)","(0.0417064373414, 0.106388825409)"
1,"(-0.0113213297804, 0.661298048381)","(-0.0222051877932, 0.390124151029)","(-0.0843919117177, 0.00106941725535)","(-0.0755602098864, 0.00340951029862)","(-0.0202991242145, 0.432097621595)","(1.0, 0.0)","(-0.0267846162338, 0.299882728041)","(-0.143812059216, 2.21543077052e-08)","(-0.131097731624, 3.48438250282e-07)","(-0.00121218169969, 0.96258617379)"
2,"(0.0190959014249, 0.459887620012)","(0.00970826453598, 0.707143753101)","(-0.0474923157426, 0.0659337607514)","(-0.0437587560729, 0.0902333647191)","(-0.183851398327, 7.1860046752e-13)","(-0.0267846162338, 0.299882728041)","(1.0, 0.0)","(-0.0745690970035, 0.00385652721617)","(-0.0410700229384, 0.111839324145)","(-0.0492479424148, 0.056529437721)"
3,"(0.0342702843992, 0.184652432528)","(0.00380659520783, 0.882890159293)","(0.0246731630186, 0.339609945091)","(-0.0416318930665, 0.107016161819)","(-0.0585099225782, 0.0234433051583)","(-0.143812059216, 2.21543077052e-08)","(-0.0745690970035, 0.00385652721617)","(1.0, 0.0)","(-0.14035870455, 4.79959317614e-08)","(-0.175080627693, 8.61100169321e-12)"
4,"(-0.0342136900742, 0.18537900548)","(0.0101913679454, 0.693292259162)","(0.0436912904085, 0.0907311410579)","(0.0614913850597, 0.0172273224942)","(-0.0504282614334, 0.0508560550686)","(-0.131097731624, 3.48438250282e-07)","(-0.0410700229384, 0.111839324145)","(-0.14035870455, 4.79959317614e-08)","(1.0, 0.0)","(-0.0273245708802, 0.290241807624)"
5,"(0.0200705424213, 0.43730098135)","(0.0121418442589, 0.638442969739)","(-0.0400330449048, 0.121189076621)","(-0.00116456763245, 0.964054761673)","(0.0417064373414, 0.106388825409)","(-0.00121218169969, 0.96258617379)","(-0.0492479424148, 0.056529437721)","(-0.175080627693, 8.61100169321e-12)","(-0.0273245708802, 0.290241807624)","(1.0, 0.0)"


In [205]:
threshold = 0.10
def color_significant_red(val):
    global threshold
    if val[0] > 0.99999: # Same index
        color = 'green'
    else:
        color = 'red' if abs(val[0]) > threshold else 'black'
    return 'color: %s' % color

colored_table = significances.style.applymap(color_significant_red)
colored_table = colored_table.format(lambda x: "{0:.3f},{1:.3f}".format(x[0],x[1]))

# Correlation Matrix
## Rows and columns are queue positions, the value is (correlation, p-value) where the p-value is Pearson's correlation
### Highlighted red are those that have correlation > 0.1

In [206]:
writer = pd.ExcelWriter('output.xlsx')
colored_table.to_excel(writer,'Sheet1')
writer.save()

In [207]:
colored_table

,-5,-4,-3,-2,-1,1,2,3,4,5
-5,"1.000,0.000","0.067,0.010","-0.156,0.000","-0.068,0.008","-0.018,0.480","-0.011,0.661","0.019,0.460","0.034,0.185","-0.034,0.185","0.020,0.437"
-4,"0.067,0.010","1.000,0.000","-0.091,0.000","-0.087,0.001","0.039,0.133","-0.022,0.390","0.010,0.707","0.004,0.883","0.010,0.693","0.012,0.638"
-3,"-0.156,0.000","-0.091,0.000","1.000,0.000","-0.101,0.000","-0.043,0.098","-0.084,0.001","-0.047,0.066","0.025,0.340","0.044,0.091","-0.040,0.121"
-2,"-0.068,0.008","-0.087,0.001","-0.101,0.000","1.000,0.000","-0.160,0.000","-0.076,0.003","-0.044,0.090","-0.042,0.107","0.061,0.017","-0.001,0.964"
-1,"-0.018,0.480","0.039,0.133","-0.043,0.098","-0.160,0.000","1.000,0.000","-0.020,0.432","-0.184,0.000","-0.059,0.023","-0.050,0.051","0.042,0.106"
1,"-0.011,0.661","-0.022,0.390","-0.084,0.001","-0.076,0.003","-0.020,0.432","1.000,0.000","-0.027,0.300","-0.144,0.000","-0.131,0.000","-0.001,0.963"
2,"0.019,0.460","0.010,0.707","-0.047,0.066","-0.044,0.090","-0.184,0.000","-0.027,0.300","1.000,0.000","-0.075,0.004","-0.041,0.112","-0.049,0.057"
3,"0.034,0.185","0.004,0.883","0.025,0.340","-0.042,0.107","-0.059,0.023","-0.144,0.000","-0.075,0.004","1.000,0.000","-0.140,0.000","-0.175,0.000"
4,"-0.034,0.185","0.010,0.693","0.044,0.091","0.061,0.017","-0.050,0.051","-0.131,0.000","-0.041,0.112","-0.140,0.000","1.000,0.000","-0.027,0.290"
5,"0.020,0.437","0.012,0.638","-0.040,0.121","-0.001,0.964","0.042,0.106","-0.001,0.963","-0.049,0.057","-0.175,0.000","-0.027,0.290","1.000,0.000"
